### Imports modules and libs

In [1]:
import pandas as pd 
import numpy as np
from ConvertFile import Convert_File_to_pd
from DataAnalyze import DataAnalyze
from AnalyzeDuplicatedData import AnalyzeDuplicatedData
from DataTransform import DataTransform
import re 
from cmath import isnan
from LoadCsvs import LoadCsvs


### Transform CSVs to DFs

In [2]:
Data_frame=Convert_File_to_pd(file_name='Match',name_table='Data',folder_name='ETL-Data-FIFA')

In [3]:
Df=Data_frame.convert_to_dataframe

### Exploratory analysis to investigate the possibility of data cleaning

In [4]:
#Imprimindo o DataFrame
Df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Premises

In [5]:
#Como são 115 colunas, é necessário prioriza-las 

for col in Df.columns:
    print(col,end=' , ')

id , country_id , league_id , season , stage , date , match_api_id , home_team_api_id , away_team_api_id , home_team_goal , away_team_goal , home_player_X1 , home_player_X2 , home_player_X3 , home_player_X4 , home_player_X5 , home_player_X6 , home_player_X7 , home_player_X8 , home_player_X9 , home_player_X10 , home_player_X11 , away_player_X1 , away_player_X2 , away_player_X3 , away_player_X4 , away_player_X5 , away_player_X6 , away_player_X7 , away_player_X8 , away_player_X9 , away_player_X10 , away_player_X11 , home_player_Y1 , home_player_Y2 , home_player_Y3 , home_player_Y4 , home_player_Y5 , home_player_Y6 , home_player_Y7 , home_player_Y8 , home_player_Y9 , home_player_Y10 , home_player_Y11 , away_player_Y1 , away_player_Y2 , away_player_Y3 , away_player_Y4 , away_player_Y5 , away_player_Y6 , away_player_Y7 , away_player_Y8 , away_player_Y9 , away_player_Y10 , away_player_Y11 , home_player_1 , home_player_2 , home_player_3 , home_player_4 , home_player_5 , home_player_6 , home_pl

In [6]:
#Para dar prioridade, os campos dos players, assim como as siglas contendo as 3 letras maiusculas serão removidos, pois tem campos acima que retraram 
#estatisticas de partidas, assim como os ids que irão servir para referenciar outras tabelas 

#Para o caso dos players, criou-se dois vetores que logicamente terão os mesmos tamanhos, já que o número dos jogadores de casa devem ser os mesmos dos visitantes
ls_col=Df.columns.tolist()
arra_home_p=list(Df.columns.str.startswith('home_player_'))
arra_away_p=list(Df.columns.str.startswith('away_player_'))

#Olhando para lista acima, percebe-se que o padrão é de 3 letras maisculas ou 4 com 365
#Seria possível de fazer uma reg expression para detectar, mas para ficar mais simples é só identificar o indice do B365H, que depois dele todos serão evitados
#Como existem Falses nas condições lógicas abaixo, esse vetor original considerado sera deslocado, logo subtraindo os Trues, teremos o vetor filtrado

ls_fin=[ls_col[a] for a in range(len(arra_home_p)) if arra_home_p[a]==False and arra_away_p[a]==False][:ls_col.index('B365H')-sum(arra_home_p+arra_away_p)]


In [7]:
dici_updated={}

for s in ls_fin:
    v=Df[s]
    dici_updated[s]=v

Df=pd.DataFrame(dici_updated)


In [8]:
#Classe responsável pelas análises exploratórias
Results_analysis=DataAnalyze(Df)

#### Detect NaNs

In [9]:
#Avaliando a proporção de NaNs em todos os campos
Results_analysis.count_nans(name_columns=list(Df.columns))


{'id': 0,
 'country_id': 0,
 'league_id': 0,
 'season': 0,
 'stage': 0,
 'date': 0,
 'match_api_id': 0,
 'home_team_api_id': 0,
 'away_team_api_id': 0,
 'home_team_goal': 0,
 'away_team_goal': 0,
 'goal': 45.27502983178721,
 'shoton': 45.27502983178721,
 'shotoff': 45.27502983178721,
 'foulcommit': 45.27502983178721,
 'card': 45.27502983178721,
 'cross': 45.27502983178721,
 'corner': 45.27502983178721,
 'possession': 45.27502983178721}

##### Observations

In [10]:
#Percebe-se que as ultimas colunas referentes aos acontecimentos da partida tem o mesmo valor da proporção de NaNs


In [11]:
##visando investigar se os dados em não NaN são os mesmos
#Como a coluna goal já existe como info em outros campos anteriores, será avaliado para os demais campos
#Fora isso, pegaremos somente as 4 últimas colunas para não gerar ainda mais complexidade nas futuras inferências
Df.drop(['goal','shoton','shotoff'],axis=1,inplace=True)



dici_not_nans={}
r=list(Df.columns)[11:]
for t in r:
    temp=Df[t].notna()
    q=[Df[t][x] for x in range(temp.shape[0]) if temp[x]]
    dici_not_nans[t]=q


Df_temp=pd.DataFrame(dici_not_nans)



In [12]:
Df_temp

,foulcommit,card,cross,corner,possession
0,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><event...
1,<foulcommit><value><stats><foulscommitted>1</f...,<card />,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>65</comment><event...
2,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...
3,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>50</comment><event...
4,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>51</comment><event...
...,...,...,...,...,...
14212,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
14213,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
14214,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />
14215,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />


##### criação de novas colunas com base nas antigas

###### COLUNA FOUL

In [13]:
#percebe-se abaixo que é possível extrair informações posteriores a respeito de tempo, o time em que executou algo e os jogadores
#Todavia, para não estabelecer mais complexidade à tabela, os dados serão ramificados em outra(s) colunas, seguindo as estratégias:


In [14]:
#Função para detectar a quantidade de ocorrencias de um time de casa ou time de fora

def cont_by_teams(Df,name_of_column,name_key_home_team,name_key_away_team):
    ds={}
    for z in range((Df.shape[0])):
        if isinstance(Df[name_of_column][z],str):
            home_team=Df['home_team_api_id'][z]
            away_team=Df['away_team_api_id'][z]
            
      

            rgxx_team="((<team>)[0-9]+(<\/team>))"

            string_aval=re.findall(rgxx_team,Df[name_of_column][z])
            
            
            cont_home,cont_away=0,0
            for s in string_aval:

                if s != None:
                    comp1=len(s[0])
                    value=s[0][len("<team>"):comp1-len("</team>")]
                   
                
                    if value == str(home_team): cont_home+=1
                    else: cont_away+=1
                
            ds[z]={name_key_home_team:cont_home,name_key_away_team:cont_away}
        
        else:
            ds[z]={name_key_home_team:0,name_key_away_team:0}


    return ds 

In [15]:
#Para coluna foulcommit, ramificação em duas colunas da quantidade de faltas cometidas pelo time da casa e pelo time visitante

dict_foul=cont_by_teams(Df=Df,name_of_column='foulcommit',name_key_home_team="quant_home_team_foul_commit",name_key_away_team="quant_away_team_foul_commit")

In [16]:
lis_foul=list(dict_foul.values())
home_list,away_list=[],[]

for i in lis_foul:
    home_list.append(i['quant_home_team_foul_commit'])
    away_list.append(i['quant_away_team_foul_commit'])

#gerar a lista contendo os valores dos times 
Df['quant_home_team_foul_commit']=home_list
Df['quant_away_team_foul_commit']=away_list


In [17]:
#Dropar a coluna antiga
Df.drop(['foulcommit'],axis=1,inplace=True)

###### COLUNA CARD

In [18]:
#Para coluna card, as colunas serão convertidas em contagem de cartões amarelos e vermelhos para os times da casa e de fora
# não é possível usar cont_by_teams como função, uma vez que as contagens não só sobre ocorrencias únicas, já que são dois tipos de cartão(amarelo e verm.)

ds_card={}

for z in range((Df.shape[0])):
    if isinstance(Df['card'][z],str):
        home_team=Df['home_team_api_id'][z]
        away_team=Df['away_team_api_id'][z]
        
        string_aval=Df['card'][z]

        rgxx="((<card_type>)[a-z-A-Z]+(<\/card_type>))"
        rgxx_team="((<team>)[0-9]+(<\/team>))"

        
        pat1=re.findall(rgxx_team,string_aval)
        pat2=re.findall(rgxx,string_aval)

        yhteam,yateam=0,0
        rhteam,rateam=0,0
        for patt1,patt2 in zip(pat1,pat2):
            if patt1!=None:

                #Para retirar o id do time no qual realizou a falta para cometer o cartão, o slice é feito a partir do comprimento da primeira tag (<team>)
                #na qual contem 6 caracteres, e a partir dele pegar o número(ID do time) até o comprimento - 7(valor de comprimento da tag </team>)
                #já que len(<tag>)+len(id_num)+len(</tag>)=len(tot)
                comp1=len(patt1[0])
                slice_pat1=patt1[0][len("<team>"):comp1-len("</team>")]

                #mesma logica para a tag card_type
                comp2=len(patt2[0])
                slice_pat2=patt2[0][len("<card_type>"):comp2-len("</card_type>")]

                if slice_pat2.lower() == 'y':
                    if slice_pat1==str(home_team):
                        yhteam+=1
                    else:yateam+=1

                else:
                    if slice_pat1==str(home_team):
                        rhteam+=1
                    else:rateam+=1
        
        
        ds_card[z]={"number_yellow_cards_home_team":yhteam,"number_yellow_cards_away_team":yateam,"number_red_cards_home_team":rhteam,"number_red_cards_away_team":rateam}

    else:
        ds_card[z]={"number_yellow_cards_home_team":0,"number_yellow_cards_away_team":0,"number_red_cards_home_team":0,"number_red_cards_away_team":0}




    

In [19]:
lis_card=list(ds_card.values())
y_home_list,y_away_list,r_home_list,r_away_list=[],[],[],[]

for i in lis_card:
    y_home_list.append(i["number_yellow_cards_home_team"])
    y_away_list.append(i["number_yellow_cards_away_team"])
    r_home_list.append(i["number_red_cards_home_team"])
    r_away_list.append(i["number_red_cards_away_team"])

#gerar a lista contendo os valores dos times 
Df["number_yellow_cards_home_team"]=y_home_list
Df["number_yellow_cards_away_team"]=y_away_list
Df["number_red_cards_home_team"]=r_home_list
Df["number_red_cards_away_team"]=r_away_list


In [20]:
#Dropar coluna antiga
Df.drop(['card'],axis=1,inplace=True)

###### COLUNA CROSS

In [21]:
#Para a coluna cross, quantidade de cruzamentos do time da casa e de fora serão convertidos em colunas 

dict_cross=cont_by_teams(Df=Df,name_of_column='cross',name_key_home_team="quant_home_team_cross",name_key_away_team="quant_away_team_cross")

In [22]:
lis_cross=list(dict_cross.values())
cross_home,cross_away=[],[]

for i in lis_cross:
   cross_home.append(i["quant_home_team_cross"])
   cross_away.append(i["quant_away_team_cross"])

#gerar a lista contendo os valores dos times 

Df["quant_home_team_cross"]=cross_home
Df["quant_away_team_cross"]=cross_away


In [23]:
#Dropar coluna antiga
Df.drop(['cross'],axis=1,inplace=True)

###### COLUNA CORNER 

In [24]:
#Para a coluna corner, quantidade de escanteios do time da casa e de fora serão convertidos em colunas 

dict_corner=cont_by_teams(Df=Df,name_of_column='corner',name_key_home_team="quant_home_team_corner",name_key_away_team="quant_away_team_corner")

In [25]:
lis_corner=list(dict_corner.values())
corner_home,corner_away=[],[]

for i in lis_corner:
   corner_home.append(i["quant_home_team_corner"])
   corner_away.append(i["quant_away_team_corner"])

#gerar a lista contendo os valores dos times 

Df["quant_home_team_corner"]=corner_home
Df["quant_away_team_corner"]=corner_away


In [26]:
#Dropar coluna antiga
Df.drop(['corner'],axis=1,inplace=True)

###### COLUNA POSSESSION

In [27]:
#Assim como card, a formação das novas colunas para essa é atípica, haja vista que a expressão regular analisada não é do id do time, mas sim 
#a posse de bola já com a tag fornecendo se é o home team ou away team
#para isso, foi feito uma média em cima dos dados de cada string em cada linha 


dici_poss={}
for i in range (Df.shape[0]):
    if isinstance(Df['possession'][i],str):
        regex="((<homepos>)[0-9]+(<\/homepos>))"
        patt_aval=re.findall(regex,Df['possession'][i])

        med_home,med_away,counter=0,0,0
        for patt in patt_aval:
            if patt != None:
                comp=len(patt[0])
                slice_pat=patt[0][len("<homepos>"):comp-len("</homepos>")]
                med_home+=int(slice_pat)
                med_away+=(100-int(slice_pat))
                counter+=1
        
        if counter !=0:
            dici_poss[i]={"avg_possession_home_team":(med_home/counter),"avg_possession_away_team":(med_away/counter)}
        else:
            dici_poss[i]={"avg_possession_home_team":0,"avg_possession_away_team":0}
            
    else:
        
        dici_poss[i]={"avg_possession_home_team":0,"avg_possession_away_team":0}









In [28]:
lis_poss=list(dici_poss.values())
poss_home,poss_away=[],[]

for i in lis_poss:
   poss_home.append(i["avg_possession_home_team"])
   poss_away.append(i["avg_possession_away_team"])

#gerar a lista contendo os valores dos times 

Df["avg_possession_home_team"]=poss_home
Df["avg_possession_away_team"]=poss_away

In [29]:
#Dropar coluna antiga
Df.drop(['possession'],axis=1,inplace=True)

In [30]:
#Por fim, o Df ficou dessa forma 
Df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,number_yellow_cards_home_team,number_yellow_cards_away_team,number_red_cards_home_team,number_red_cards_away_team,quant_home_team_cross,quant_away_team_cross,quant_home_team_corner,quant_away_team_corner,avg_possession_home_team,avg_possession_away_team
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,0,0,0,0,0,0,0,0,0.0,0.0
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,0,0,0,0,0,0,0,0,0.0,0.0
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,0,0,0,0,0,0,0,0,0.0,0.0
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,0,0,0,0,0,0,0,0,0.0,0.0
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,0,0,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,0,0,0,0,0,0,0,0,0.0,0.0
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,0,0,0,0,0,0,0,0,0.0,0.0
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,0,0,0,0,0,0,0,0,0.0,0.0
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,0,0,0,0,0,0,0,0,0.0,0.0


#### Detect Data Duplicated

In [31]:
#detecção de duplicatas nos campos onde não podem haver repetições 

#Id não pode haver repetição, uma vez que é único
Result_dup_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='id')

#Para a coluna country_id, pode haver repetição, já que partidas diferentes podem ocorrer no mesmo país(1->n)
#Da mesma forma para league_id

#No entanto, match_api_id não pode haver ocorrência de repetição, já que o id da partida via API
Result_dup_match_api_id=AnalyzeDuplicatedData(dataframe=Df,name_col_data='match_api_id')



In [32]:
#Avaliando a quantidade de ids duplicados
Result_dup_id.print_duplicated_data(table_name='match')
Result_dup_match_api_id.print_duplicated_data(table_name='match')



a quantidade de dados duplicados da tabela match  na coluna id são 0
a quantidade de dados duplicados da tabela match  na coluna match_api_id são 0


#### Identify Outliers

In [33]:
#pela lógica o número de cartões vermelhos não pode ser acima de 11, uma vez que só tem 11 jogadores
Results_analysis=DataAnalyze(Df)

Results_analysis.identify_outliers_IQR(name_column="number_red_cards_home_team")



[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [34]:
Results_analysis.identify_outliers_IQR(name_column="number_red_cards_away_team")

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,


In [35]:
#Como há um grande volume de números 0, praticamente metade dos dados dessa coluna, então as tendências para valores de dispersão acabam afetando bastante
#por isso o 1 ou 2 foram detectados, mesmo sendo possível 
#Mesmo fazendo para outros métodos como é o Z-score modificado, presente na classe DataAnalyze, identificou-se esses mesmos valores



### Analyze  typing

In [36]:
#analisando a tipagem esperada de forma geral
Df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             25979 non-null  int64  
 1   country_id                     25979 non-null  int64  
 2   league_id                      25979 non-null  int64  
 3   season                         25979 non-null  object 
 4   stage                          25979 non-null  int64  
 5   date                           25979 non-null  object 
 6   match_api_id                   25979 non-null  int64  
 7   home_team_api_id               25979 non-null  int64  
 8   away_team_api_id               25979 non-null  int64  
 9   home_team_goal                 25979 non-null  int64  
 10  away_team_goal                 25979 non-null  int64  
 11  quant_home_team_foul_commit    25979 non-null  int64  
 12  quant_away_team_foul_commit    25979 non-null 

In [37]:
#Os dados encontram-se todos compatíveis com a eventual tipagem em que eles serão carregados no bd
#Porém, é necessário fazer uma validação sobre se a estrutura da data está coerente, já que object pode contemplar tudo

Results_analysis.analyze_typing_expected_timestamp(name_column='date')


'all the data are in the date structure'

### Data transform 

In [38]:
#Como não houve dados NaN e dados duplicados em colunas nas quais não podem permití-los 
#Não é preciso removê-los
#Assim como a tipagem, todos dados encontram-se coerentes

#No entanto, é necessário fazer um teste de sanidade para avaliar se as novas colunas formadas através das tags estão coerentes



#### Sanity test

In [39]:
# Começando com a coluna possession, é impossível ter essa coluna com dados 0, porque todo jogo irá gerar uma possessão de bola entre os times 

#dessa forma,
Df_transf=Df.copy()

In [40]:
possession_nulls=Df.loc[(Df['avg_possession_home_team']==0) | (Df['avg_possession_away_team']==0)]

In [41]:
Df_transf=Df_transf.drop(possession_nulls.index)

In [42]:
#Outro ponto importante é se a quantidade de cartões amarelos ou vermelhos existirem e não houverem faltas cometidas
#Para  o caso dos cartões vermelhos 

Df_transf[(Df_transf['number_red_cards_away_team'] > (Df_transf['quant_away_team_foul_commit'])) | (Df_transf['number_red_cards_home_team'] > Df_transf['quant_home_team_foul_commit']) ]


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,number_yellow_cards_home_team,number_yellow_cards_away_team,number_red_cards_home_team,number_red_cards_away_team,quant_home_team_cross,quant_away_team_cross,quant_home_team_corner,quant_away_team_corner,avg_possession_home_team,avg_possession_away_team


In [43]:
#Para  o caso dos cartões amarelos 
Df_transf[(Df_transf['number_yellow_cards_away_team'] > (Df_transf['quant_away_team_foul_commit'])) | (Df_transf['number_yellow_cards_home_team'] > Df_transf['quant_home_team_foul_commit']) ]

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,number_yellow_cards_home_team,number_yellow_cards_away_team,number_red_cards_home_team,number_red_cards_away_team,quant_home_team_cross,quant_away_team_cross,quant_home_team_corner,quant_away_team_corner,avg_possession_home_team,avg_possession_away_team
22792,22793,21518,21518,2011/2012,21,2012-01-29 00:00:00,1051844,8603,7878,1,...,3,6,0,0,18,21,11,6,45.50,54.50
22809,22810,21518,21518,2011/2012,23,2012-02-11 00:00:00,1051861,8371,8634,3,...,2,6,0,0,8,24,3,12,33.50,66.50
23033,23034,21518,21518,2011/2012,9,2011-10-22 00:00:00,1051735,8634,8302,0,...,2,7,0,1,28,7,13,3,73.25,26.75


In [44]:
#Porém, é valido ressaltar que podem ocorrer penalizações através de cartão amarelo sem que haja falta
# essa análise de sanidade neste caso foi feita para saber se houvesse uma discrepância grande entre esses casos, pois poderia existir alguma inconstância nos dados 

### Load CSV

In [45]:
Df_load=LoadCsvs(dataframe=Df_transf,new_folder='Load_csvs',name_dataframe='Matchsss',folder_name=Data_frame.folder_name)

In [46]:
Df_load.load_csv